# Analysis 0: Preprocessing
- Drop missing variables with > 60% missingness
- Demographics table 

## Import libraries

In [16]:
%load_ext rpy2.ipython 
# Load the R magic extension

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import os
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import patsy
from lifelines import CoxPHFitter 
import statsmodels as sm
from pathlib import Path

In [18]:
# Add the directory to sys.path
import sys
module_path = Path('./../code')
sys.path.append(str(module_path))
import utils

In [19]:
# Import libraries to allow data to be passed between Python and R env
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects import r

pandas2ri.activate()

In [20]:
%%R
# Run this cell to install packages the first time. 

# install.packages("bshazard")
# install.packages("survival")
# install.packages("tidyr")
# install.packages("dplyr")
# install.packages("magrittr")
# install.packages("tableone")
# install.packages("pROC")
# install.packages("PRROC")
# install.packages("caret")
# install.packages("survivalROC")
# install.packages("survminer")
# install.packages("scales")
# install.packages("broom")
# install.packages("purrr")


NULL


In [21]:
%%R
library(bshazard)
library(survival)
library(tidyr)
library(dplyr)
library(magrittr)
library(tableone)
library(pROC)
library(PRROC)
library(caret)
library(survivalROC)
library(survminer)
library(scales)
library(finalfit)
library(broom)
library(broom.helpers)
library(purrr)

In [22]:
import warnings
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)

# Displays all the columns, does 
pd.set_option('display.max_columns', None)

## Import data

In [23]:
demographics_table_filename = './../results/demographics_table.csv'
univariate_filename = './../results/univariate_analysis.csv'
multivariate_filename = './../results/multivariate_analysis.csv'

In [24]:
data_filename = './../data/cleaned_cohort_20250429.csv'
df = pd.read_csv(data_filename)[utils.VARS_TO_ANALYZE]
df.shape

(1745288, 79)

In [25]:
df.ugica.value_counts()

ugica
0.0    1744975
1.0        313
Name: count, dtype: int64

In [26]:
df.hpylori_active_chronic_binary.value_counts()

hpylori_active_chronic_binary
0    1743688
1       1600
Name: count, dtype: int64

In [27]:
# Pass the DataFrame into the R environment
def pass_df(df, r_df_name):
    ro.globalenv[r_df_name] = df

ro.globalenv['numerical_vars'] = utils.NUMERICAL_VARS
ro.globalenv['categorical_vars'] = utils.CATEGORICAL_VARS # + ['sex_clean']
ro.globalenv['demographics_table_filename'] = str(demographics_table_filename)
ro.globalenv['univariate_filename'] = str(univariate_filename)
ro.globalenv['multivariate_filename'] = str(multivariate_filename)

##### Create buckets to analyze stratified groups 

In [28]:
def bucket_visit_year(row):
    visit_year = row.visit_year

    if 2011 <= visit_year <= 2014:
        return "2011-2014"
    elif 2015 <= visit_year <= 2018:
        return "2015-2018"
    elif visit_year >= 2019:
        return "2019-2022"
    
def bucket_age(row):
    age = row.age

    if 40 <= age < 50:
        return "40-49"
    elif 50 <= age < 60:
        return "50-59"
    elif 60 <= age < 70:
        return "60-69"
    elif 70 <= age < 80:
        return "70-79"
    elif 80 <= age <= 85:
        return "80-85"
    else:
        return "Out of range"


In [29]:
df.loc[:, 'visit_year_bucket'] = df.apply(bucket_visit_year, axis=1)
df.loc[:, 'age_bucket'] = df.apply(bucket_age, axis=1)

In [30]:
pd.crosstab(df.visit_year_bucket, df.ugica)

ugica,0.0,1.0
visit_year_bucket,,
2011-2014,314313,153
2015-2018,702096,142
2019-2022,728566,18


In [31]:
pd.crosstab(df.age_bucket, df.ugica)

ugica,0.0,1.0
age_bucket,,
40-49,531082,27
50-59,467977,59
60-69,407387,116
70-79,255518,85
80-85,83002,26
Out of range,9,0


In [32]:
df[df.age.isna()].shape

(9, 81)

##### Add variables to compare with current guidelines for risk-factor triggered screening for EAC

In [33]:
def num_risk_factors(row):
    score = 0 

    if row.age > 50: 
        score += 1 
    if row.sex == 'MALE':
        score += 1 
    if row.race_clean == 'White':
        score += 1
    if row.tobacco_binary == '1':
        score += 1
    if row.gerd == '1':
        score += 1
    if row.BMI_baseline >= 30:
        score += 1 
    if row.famhx_esophagealca or row.famhx_barretts:
        score += 1 
    
    return score 

df['eac_risk_factors_screening'] = df.apply(lambda x: num_risk_factors(x), axis=1)
df['meets_eac_screening'] = (df.eac_risk_factors_screening >= 3).astype(int)

##### Clean subtype cancer outcomes


In [34]:
df[['ugica_ESCC', 'ugica_EAC', 'ugica_CGC', 'ugica_NCGC']] = df[['ugica_ESCC', 'ugica_EAC', 'ugica_CGC', 'ugica_NCGC']].fillna(0)

In [35]:
df[['ugica', 'ugica_ESCC', 'ugica_EAC', 'ugica_CGC', 'ugica_NCGC']].sum()

ugica         313.0
ugica_ESCC     62.0
ugica_EAC      68.0
ugica_CGC      63.0
ugica_NCGC    120.0
dtype: float64

In [36]:
pass_df(df, "r_df")

/Users/alyssa.chen/miniconda3/envs/rpy2_env/lib/python3.10/site-packages/rpy2/robjects/pandas2ri.py:65: UserWarning: Error while trying to convert the column "subtype". Fall back to string conversion. The error is: Series can only be of one type, or None (and here we have <class 'float'> and <class 'str'>). If happening with a pandas DataFrame the method infer_objects() will normalize data types before conversion.
  warnings.warn('Error while trying to convert '


## Preprocessing

In [37]:
%%R 
# Ignore these columns
cols_to_ignore <- c(
    'months_to_event', 'ugica', 'ugica_ESCC', 'ugica_EAC', 'ugica_CGC', 'ugica_NCGC', 
    'death', 'subtype', 'visit_year', 'diagnosis_year', 'encounter_type', 'social_language', 
    'days_to_event', 'days_to_dx', 'days_to_death',
    "eac_risk_factors_screening", "meets_screening"
)

### Remove variables that have >60% missing

In [38]:
%%R 
missing_vars <- names(which(sapply(r_df, function(x) mean(is.na(x))) > 0.60))
missing_vars <- missing_vars[!missing_vars %in% cols_to_ignore]
missing_vars

 [1] "alcohol_all_missing"            "alcohol_binary_missing"        
 [3] "hpylori_active_missing"         "hpylori_active_chronic_missing"
 [5] "hgball_baseline"                "hgb_baseline"                  
 [7] "mcv_baseline"                   "wbc_baseline"                  
 [9] "plt_baseline"                   "sodium_baseline"               
[11] "potassium_baseline"             "chloride_baseline"             
[13] "bicarbonate_baseline"           "bun_baseline"                  
[15] "scr_baseline"                   "magnesium_baseline"            
[17] "calcium_baseline"               "phosphate_baseline"            
[19] "ast_baseline"                   "alt_baseline"                  
[21] "alp_baseline"                   "tbili_baseline"                
[23] "tprotein_baseline"              "albumin_baseline"              
[25] "tsh_baseline"                   "vitD_baseline"                 
[27] "triglycerides_baseline"         "LDL_baseline"                  
[29] "

In [39]:
%%R 
print(dim(r_df))
r_df_nonmissing <- r_df[, !names(r_df) %in% missing_vars]
print(dim(r_df_nonmissing))

[1] 1745288      83
[1] 1745288      54


### Impute mean for continuous variables with <= 60% missing

In [40]:
%%R 
# Get the variables that have missing data 
missing_less_60_vars <- names(which(sapply(r_df_nonmissing, function(x) mean(is.na(x))) > 0))

# Filter to only include variables in continuous_vars
cols_to_impute <- missing_less_60_vars[
  (missing_less_60_vars %in% numerical_vars) & 
  !(missing_less_60_vars %in% cols_to_ignore)
]
cols_to_impute

[1] "age"              "height_baseline"  "weight_baseline"  "BMI_baseline_all"
[5] "BMI_baseline"    


In [41]:
%%R 
r_df_imputed <- r_df_nonmissing %>%
  mutate(across(all_of(cols_to_impute), ~ ifelse(is.na(.), mean(., na.rm = TRUE), .)))

In [42]:
%%R 
# Before imputation
summary(r_df_nonmissing[, cols_to_impute])

 [1] "Min.   :40.00  "   "1st Qu.:47.00  "   "Median :57.00  "  
 [4] "Mean   :57.71  "   "3rd Qu.:67.00  "   "Max.   :85.00  "  
 [7] "NA's   :9  "       "Min.   :  1.6  "   "1st Qu.: 63.0  "  
[10] "Median : 66.0  "   "Mean   : 66.0  "   "3rd Qu.: 69.0  "  
[13] "Max.   :115.0  "   "NA's   :766584  "  "Min.   :    4  "  
[16] "1st Qu.: 2304  "   "Median : 2720  "   "Mean   : 2794  "  
[19] "3rd Qu.: 3184  "   "Max.   :23648  "   "NA's   :714568  " 
[22] "Min.   :    0.0  " "1st Qu.:   23.8  " "Median :   27.1  "
[25] "Mean   :   31.5  " "3rd Qu.:   31.2  " "Max.   :48330.8  "
[28] "NA's   :820984  "  "Min.   :    0.0  " "1st Qu.:   23.8  "
[31] "Median :   27.2  " "Mean   :   32.6  " "3rd Qu.:   31.2  "
[34] "Max.   :48307.3  " "NA's   :836779  " 


In [43]:
%%R 
# After imputation
summary(r_df_imputed[, cols_to_impute])

 [1] "Min.   :40.00  "    "1st Qu.:47.00  "    "Median :57.00  "   
 [4] "Mean   :57.71  "    "3rd Qu.:67.00  "    "Max.   :85.00  "   
 [7] "Min.   :  1.61  "   "1st Qu.: 65.00  "   "Median : 65.99  "  
[10] "Mean   : 65.99  "   "3rd Qu.: 66.15  "   "Max.   :115.00  "  
[13] "Min.   :    3.98  " "1st Qu.: 2576.00  " "Median : 2793.47  "
[16] "Mean   : 2793.47  " "3rd Qu.: 2836.80  " "Max.   :23648.00  "
[19] "Min.   :    0.04  " "1st Qu.:   26.64  " "Median :   31.53  "
[22] "Mean   :   31.53  " "3rd Qu.:   31.53  " "Max.   :48330.77  "
[25] "Min.   :    0.00  " "1st Qu.:   26.80  " "Median :   32.63  "
[28] "Mean   :   32.63  " "3rd Qu.:   32.63  " "Max.   :48307.30  "


### Normalize continuous variables

In [44]:
%%R 
vars_to_normalize <- names(r_df_imputed)[
    !names(r_df_imputed) %in% cols_to_ignore &
    names(r_df_imputed) %in% numerical_vars
]

preproc <- preProcess(r_df_imputed[, vars_to_normalize], method = c("center", "scale"))
r_df_normal <- predict(preproc, r_df_imputed)
dim(r_df_normal)


[1] 1745288      54


### Add outcome variables

In [56]:
%%R 
source("utils.R") # Wrote this code later in the analysis in R 

r_df_normal$ugica_5yr <- count_event(r_df_normal, "ugica")
r_df_normal$escc_5yr <- count_event(r_df_normal, "ugica_ESCC")
r_df_normal$eac_5yr <- count_event(r_df_normal, "ugica_EAC")
r_df_normal$cgc_5yr <- count_event(r_df_normal, "ugica_CGC")
r_df_normal$ncgc_5yr <- count_event(r_df_normal, "ugica_NCGC")

r_df_normal$ugica_1yr <- count_event(r_df_normal, "ugica", horizon_months = 12)
r_df_normal$escc_1yr <- count_event(r_df_normal, "ugica_ESCC", horizon_months = 12)
r_df_normal$eac_1yr <- count_event(r_df_normal, "ugica_EAC", horizon_months = 12)
r_df_normal$cgc_1yr <- count_event(r_df_normal, "ugica_CGC", horizon_months = 12)
r_df_normal$ncgc_1yr <- count_event(r_df_normal, "ugica_NCGC", horizon_months = 12)

r_df_normal$ugica_3yr <- count_event(r_df_normal, "ugica", horizon_months = 36)
r_df_normal$escc_3yr <- count_event(r_df_normal, "ugica_ESCC", horizon_months = 36)
r_df_normal$eac_3yr <- count_event(r_df_normal, "ugica_EAC", horizon_months = 36)
r_df_normal$cgc_3yr <- count_event(r_df_normal, "ugica_CGC", horizon_months = 36)
r_df_normal$ncgc_3yr <- count_event(r_df_normal, "ugica_NCGC", horizon_months = 36)

r_df_normal$ugica_10yr <- count_event(r_df_normal, "ugica", horizon_months = 120)
r_df_normal$escc_10yr <- count_event(r_df_normal, "ugica_ESCC", horizon_months = 120)
r_df_normal$eac_10yr <- count_event(r_df_normal, "ugica_EAC", horizon_months = 120)
r_df_normal$cgc_10yr <- count_event(r_df_normal, "ugica_CGC", horizon_months = 120)
r_df_normal$ncgc_10yr <- count_event(r_df_normal, "ugica_NCGC", horizon_months = 120)

In [75]:
%%R
# Display value counts for each new column
new_columns <- c(
    'ugica', 'ugica_10yr', 'ugica_5yr', 'ugica_3yr', 'ugica_1yr',
    'ugica_ESCC', 'escc_10yr', 'escc_5yr', 'escc_3yr', 'escc_1yr',
    'ugica_EAC', 'eac_10yr', 'eac_5yr', 'eac_3yr', 'eac_1yr',
    'ugica_CGC', 'cgc_10yr', 'cgc_5yr', 'cgc_3yr', 'cgc_1yr',
    'ugica_NCGC', 'ncgc_10yr', 'ncgc_5yr', 'ncgc_3yr', 'ncgc_1yr'
)

# Convert the table frame to a dataframe
value_counts_df <- do.call(rbind, lapply(new_columns, function(col) {
    data.frame(Column = col, Cancer = names(table(r_df_normal[[col]])), Count = as.vector(table(r_df_normal[[col]])))
}))


In [76]:
%%R 
value_counts_df[value_counts_df$Cancer == 1, ]

       Column Cancer Count
2       ugica      1   313
4  ugica_10yr      1   293
6   ugica_5yr      1   185
8   ugica_3yr      1    91
11 ugica_ESCC      1    62
13  escc_10yr      1    52
15   escc_5yr      1    28
17   escc_3yr      1    11
20  ugica_EAC      1    68
22   eac_10yr      1    67
24    eac_5yr      1    45
26    eac_3yr      1    19
29  ugica_CGC      1    63
31   cgc_10yr      1    61
33    cgc_5yr      1    37
35    cgc_3yr      1    15
38 ugica_NCGC      1   120
40  ncgc_10yr      1   113
42   ncgc_5yr      1    75
44   ncgc_3yr      1    46


## Demographics table

In [77]:
%%R 
#vars_to_analyze <- unlist(c(categorical_vars, numerical_vars))

demtable <- CreateTableOne(
    #vars = vars_to_analyze,
    data = r_df_nonmissing,
    factorVars = unlist(categorical_vars),
    strata = "ugica",
    addOverall = TRUE,
    includeNA = TRUE
)
demtable_df <- print(demtable, quote = FALSE, noSpaces = TRUE, printToggle = FALSE, missing = TRUE)

write.csv(demtable_df, file = demographics_table_filename)

In [78]:
%%R 
write.csv(r_df_normal, "df_analysis0_imputed.csv", row.names = FALSE)